#### -----------------------------------------------------------------------------<br>Copyright (c) 2022, Lucid Vision Labs, Inc.
##### THE  SOFTWARE  IS  PROVIDED  "AS IS",  WITHOUT  WARRANTY  OF  ANY  KIND,<br>EXPRESS  OR  IMPLIED,  INCLUDING  BUT  NOT  LIMITED  TO  THE  WARRANTIES<br>OF  MERCHANTABILITY,  FITNESS  FOR  A  PARTICULAR  PURPOSE  AND<br>NONINFRINGEMENT.  IN  NO  EVENT  SHALL  THE  AUTHORS  OR  COPYRIGHT  HOLDERS<br>BE  LIABLE  FOR  ANY  CLAIM,  DAMAGES  OR  OTHER  LIABILITY,  WHETHER  IN  AN<br>ACTION  OF  CONTRACT,  TORT  OR  OTHERWISE,  ARISING  FROM,  OUT  OF  OR  IN<br>CONNECTION  WITH  THE  SOFTWARE  OR  THE  USE  OR  OTHER  DEALINGS  IN  THE  SOFTWARE.<br>-----------------------------------------------------------------------------

In [ ]:
import time
from arena_api.system import system

#### Chunk Data: Introduction
>    This example introduces the configuration and retrieval of chunk data.
    Chunk data is data that has been tacked on to the end of image data in
    order to provide useful information on the image. Configuring chunk data
    involves activating chunk mode and enabling desired chunks. Retrieving
    chunk data from an image is similar to retrieving nodes from a node map.

> -	Activates chunk mode
> - Enable selected chunks
> - Starts the stream and gets images
> - Retrieves chunk data from image
> - Requeues buffers and stops the stream

In [ ]:
"""
This function waits for the user to connect a device before raising
an exception
"""

tries = 0
tries_max = 6
sleep_time_secs = 10
while tries < tries_max:  # Wait for device for 60 seconds
    devices = system.create_device()
    if not devices:
        print(
            f'Try {tries+1} of {tries_max}: waiting for {sleep_time_secs} '
            f'secs for a device to be connected!')
        for sec_count in range(sleep_time_secs):
            time.sleep(1)
            print(f'{sec_count + 1 } seconds passed ',
                  '.' * sec_count, end='\r')
        tries += 1
    else:
        print(f'Created {len(devices)} device(s)\n')
        device = devices[0]
        break
else:
    raise Exception(f'No device found! Please connect a device and run '
                    f'the example again.')

##### Store initial chunk mode value
> These initial values are restored to the device after the example is completed

In [ ]:
nodemap = device.nodemap

initial_chunk_mode_active = nodemap.get_node("ChunkModeActive").value

##### Activate chunk mode and enable selected chunks.

In [ ]:
print('Activating chunk data on device')
nodemap.get_node('ChunkModeActive').value = True

##### List of chunk names to send with the buffer and store initial values for the list
>  Use 'ChunkSelector' node to select the chunk to send. This is done
    by setting the value of 'ChunkSelector' node to the name of the chunk,
    then enabling the chunk by setting 'ChunkEnable' to 'True'. <br>
> These initial values are restored to the device after the example is completed

In [ ]:
chunks_to_append_to_buffer = ['PixelFormat', 'CRC',
                              'Width', 'Height',
                              'OffsetX', 'OffsetY']

chunk_selector_node = nodemap.get_node('ChunkSelector')
chunk_enable_node = nodemap.get_node('ChunkEnable')

initial_chunk_selector = chunk_selector_node.value
initial_chunk_enable = {}

for chunk_name in chunks_to_append_to_buffer:
    print(f'\tsetting \'ChunkSelector\' node value to \'{chunk_name}\'')
    chunk_selector_node.value = chunk_name
    initial_chunk_enable[chunk_name] = chunk_enable_node.value
    print(f'\tenabling \'{chunk_name}\' by setting \'ChunkEnable\' '
          f'node value to \'True\'')
    chunk_enable_node.value = True

##### Grab images -------------------------------------------------------------
> - Starting the stream allocates buffers, which can be passed in as
 an argument (default: 10), and begins filling them with data.
 Buffers must later be requeued to avoid memory leaks.<br>
> - 'Device.get_buffer()' with no arguments returns one buffer(NOT IN A LIST)<br>
> - 'Device.get_buffer(30)' returns 30 buffers(IN A LIST)<br>
> - 'Device.requeue_buffer()' takes a buffer or many buffers in a list or tuple

In [ ]:
with device.start_stream():
    print(f'Stream started with 10 buffers')
    
    print('\tGet chunk data buffer(s)')

    # This would timeout or returns all of the 10 buffers
    chunk_data_buffers = device.get_buffer(number_of_buffers=10)
    print(f'\t{len(chunk_data_buffers)} chunk data buffers received')

    '''
    To access the chunks that are appended to a buffer use buffer
    'Buffer.get_chunk()' method. The chunk name to get is the same as
    the chunk value that was set and enabled but preceded with 'Chunk'.
    '''
    chunk_node_names = ['ChunkPixelFormat', 'ChunkCRC',
                        'ChunkWidth', 'ChunkHeight',
                        'ChunkOffsetX', 'ChunkOffsetY']

    # Iterate over every chunk data buffer and print chunks' value
    for buffer_index, chunk_data_buffer in enumerate(chunk_data_buffers):
        '''
        Get all chunks in list from buffer
            'Buffer.get_chunk()' may raise an exception because the
            buffer is incomplete
        '''
        if chunk_data_buffer.is_incomplete:
            print(f'\t\t---------------------------------------------')
            print(f'\t\tChunk data buffer{buffer_index} is incomplete')
            print(f'\t\t---------------------------------------------')
            # Continue
        try:
            chunk_nodes_dict = chunk_data_buffer.get_chunk(
                chunk_node_names)

            # Print the value of the chunks for the current buffer
            print(f'\t\tChunk data buffer{buffer_index} chunks value:')
            for chunk_node_name, chunk_node in chunk_nodes_dict.items():
                print(f'\t\t\t{chunk_node_name} = {chunk_node.value}')
        except ValueError:
            print(f'\t\t\tFailed to get chunks')
            print(f'\t\t---------------------------------------------')
            continue

    # Requeue the chunk data buffers
    device.requeue_buffer(chunk_data_buffers)

##### When the scope of the context manager ends, then 'Device.stop_stream()' is called automatically

##### Clean up ----------------------------------------------------------------
> - Restore initial values to the device.
> - Destroy device. This call is optional and will automatically be
  called for any remaining devices when the system module is unloading.

In [ ]:
for chunk_name in chunks_to_append_to_buffer:
    chunk_selector_node.value = chunk_name
    chunk_enable_node.value = initial_chunk_enable[chunk_name]

chunk_selector_node.value = initial_chunk_selector
nodemap.get_node("ChunkModeActive").value = initial_chunk_mode_active

system.destroy_device()